In [ ]:
import os 
from google.colab import drive
drive.mount('/content/drive')

# TODO: 샌드박스명 설정
%env SANDBOX_NAME=common
# TODO: 브랜치명 설정 (lesimor/fairseq)
%env BRANCH=feature/add_colab_notebook
# TODO: 레포지토리 경로를 자신의 구글드라이브 환경에 맞게 수정해야 함
%env REPO_ROOT=/content/drive/MyDrive/repos/fairseq

REPO_ROOT=os.environ['REPO_ROOT']
os.environ["TRANSLATION_ROOT"] = f"{REPO_ROOT}/examples/translation"

sandbox_name = os.environ["SANDBOX_NAME"]
os.environ["SANDBOX_NAME"] = sandbox_name

sandbox_root = f"{REPO_ROOT}/sandbox/translation-{sandbox_name}"
os.environ["SANDBOX_ROOT"] = sandbox_root

text_root = f"{sandbox_root}/text"
os.environ["TEXT_ROOT"] = text_root
os.environ["TEXT"] = f'{text_root}/iwslt14.tokenized.de-en'

databin_root = f'{sandbox_root}/data-bin'
os.environ["DATABIN_ROOT"] = databin_root
os.environ["DATABIN"] = f'{databin_root}/iwslt14.tokenized.de-en'
os.environ["CHECKPOINT"] = f'{sandbox_root}/checkpoints'

# PYTHONPATH 설정
if not REPO_ROOT in os.environ['PYTHONPATH'].split(':'):
  os.environ['PYTHONPATH'] += f":{REPO_ROOT}" 

print(os.environ['PYTHONPATH'].split(':'))

In [ ]:
# 이미 클론받았으면 패스
![ ! -d $REPO_ROOT ] && git clone https://github.com/lesimor/fairseq.git $REPO_ROOT

%cd $REPO_ROOT
!git fetch
!git reset --hard origin/$BRANCH

In [ ]:
# 레포지토리 루트로 이동
import os
path = os.environ['REPO_ROOT']
os.chdir(path)

# fairseq 설치
!pip install --editable .

In [ ]:
# translation example 디렉토리로 이동
!mkdir -p $TEXT_ROOT

import os
path = os.environ['TEXT_ROOT']
os.chdir(path)

!bash $TRANSLATION_ROOT/prepare-iwslt14.sh

In [ ]:
# 기존 디렉토리 제거
!rm -rf $DATABIN_ROOT

# train data 자르기
! chmod +x $TRANSLATION_ROOT/cut_traindata.sh
! $TRANSLATION_ROOT/cut_traindata.sh $TEXT 100

# preprocessing
![ ! -d $DATABIN_ROOT ] && fairseq-preprocess --source-lang de --target-lang en \
    --trainpref $TEXT/train-cut --validpref $TEXT/valid --testpref $TEXT/test \
    --destdir $DATABIN




In [ ]:
# 체크포인트를 저장할 디렉토리 생성
!mkdir -p $CHECKPOINT

! fairseq-train $DATABIN \
    --arch transformer_iwslt_de_en --save-dir $CHECKPOINT \
    --max-epoch 10 --max-tokens 4000 --optimizer adam

In [ ]:
! python $REPO_ROOT/scripts/average_checkpoints.py \
    --inputs $CHECKPOINT \
    --num-epoch-checkpoints 3 \
    --output $CHECKPOINT/checkpoint.avg3.ptver

In [ ]:
! fairseq-generate $DATABIN \
  --path $CHECKPOINT/checkpoint1.pt \
  --beam 5 --batch-size 128

In [ ]:
! fairseq-generate \
    $DATABIN \
    --path $CHECKPOINT/checkpoint.avg3.ptver \
    --beam 4 --lenpen 0.6 --remove-bpe > $CHECKPOINT/gen.out